In [2]:
!pip install pick

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [8]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('//content/drive/MyDrive/week0_starter_network_analysis')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

In [35]:
data_loader = SlackDataLoader('/content/drive/MyDrive/Anonymized_B6SlackExport_25Nov23/anonymized')
# Load data from a Slack channel
channels = data_loader.get_channels()
users = data_loader.get_users()


In [36]:
print(channels)

[{'id': 'C03T0APHX63', 'name': 'all-community-building', 'created': 1660301317, 'creator': 'U03TEPYRM2P', 'is_archived': False, 'is_general': False, 'members': ['U03T89ACUUW', 'U03TEPYRM2P', 'U03TNP8Q8CT', 'U03TT5KEYCF', 'U03TX2VN6H5', 'U03U1FNPEUX', 'U03U1GHT39V', 'U03U1HAG9TR', 'U03U1J51VFZ', 'U03U4GULU3Y', 'U03U93GNNVB', 'U03U9DB7REG', 'U03U9EJR362', 'U03UAKATQ22', 'U03UD4FEDHB', 'U03UD5B7C3X', 'U03UD63A8PP', 'U03UD68RQH3', 'U03UDBUL7CL', 'U03UDKKESB1', 'U03UFV7HFNF', 'U03UG0SFHGT', 'U03UG0YHAUT', 'U03UG1Z21JP', 'U03UG32J3PC', 'U03UG4Q7V42', 'U03UG5VFN03', 'U03UH397319', 'U03UHB8CXDY', 'U03UJGFG2HJ', 'U03UJGP0C68', 'U03UJGRN5E0', 'U03UJH1EQQL', 'U03UJKJGRAQ', 'U03UJN29Y4C', 'U03UKL27B0R', 'U03UL5LSTG9', 'U03UP7V9Q57', 'U03UUMR26Q1', 'U03UUN8M4RX', 'U03UUP56MDF', 'U03UUR571A5', 'U03UUS0MZCZ', 'U03UVHCV6KB', 'U03UYNR4TS4', 'U03V1AM5TFA', 'U03V5Q9N516', 'U03V61VGQG0', 'U03V6HMRPGQ', 'U03V785NLSU', 'U03V8LHPDME'], 'topic': {'value': '', 'creator': '', 'last_set': 0}, 'purpose': {'value'

In [37]:
channel_name = "all-week8"
channel_messages = data_loader.get_channel_messages(channel_name)


In [40]:
channel_path = '/content/drive/MyDrive/Anonymized_B6SlackExport_25Nov23/anonymized/all-week9'
channel_messages_df = utils.get_messages_from_channel(channel_path)


ValueError: ignored

In [38]:
print(channel_messages)

None


### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [ ]:
def get_top_20_user(data, channel='Random'):
    """get user with the highest number of message sent to any channel"""

    data['sender_name'].value_counts()[:20].plot.bar(figsize=(15, 7.5))
    plt.title(f'Top 20 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

    data['sender_name'].value_counts()[-10:].plot.bar(figsize=(15, 7.5))
    plt.title(f'Bottom 10 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance

In [ ]:
# which user has the highest number of reply counts?

In [ ]:
# Visualize reply counts per user per channel

In [ ]:
# what is the time range of the day that most messages are sent?


In [ ]:
# what kind of messages are replied faster than others?

In [ ]:
# Relationship between # of messages and # of reactions

In [ ]:
# Classify messages into different categories such as questions, answers, comments, etc.

In [ ]:
# Which users got the most reactions?

In [ ]:
# Model topics mentioned in the channel

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?